In [10]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


In [11]:
# Veri setinin yüklenmesi
data = pd.read_csv('D:\\hepsiS1_colab.csv')

print ('*******************************************************')
# İlk birkaç satırı görüntüleme
print(data.head())
print ('*******************************************************')

# Veri setinin yapısını inceleme
print(data.info())


*******************************************************
   labels                                               text
0       0  fiyat ortalama mouse oymak almak demek standar...
1       0  ürün güzel lakin beklemek değil sağ tuş zorlam...
2       0  hesap kalite ürün satık memnun site teşekkür e...
3       0  aymak mouse kul alıcı iyi kul kolay sol tıklam...
4       0  fiyat kalite değerlendirmek güzel ürün laptop ...
*******************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243497 entries, 0 to 243496
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   labels  243497 non-null  int64 
 1   text    243443 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.7+ MB
None


In [12]:
# NaN değerlerini boş string ile değiştirme
data['text'] = data['text'].fillna('')

# Temizlenmiş veri setinin ilk birkaç satırını görüntüleme
print(data.head())


   labels                                               text
0       0  fiyat ortalama mouse oymak almak demek standar...
1       0  ürün güzel lakin beklemek değil sağ tuş zorlam...
2       0  hesap kalite ürün satık memnun site teşekkür e...
3       0  aymak mouse kul alıcı iyi kul kolay sol tıklam...
4       0  fiyat kalite değerlendirmek güzel ürün laptop ...


In [13]:
def clean_text(text):
    # URL'leri kaldır
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # İşaretlemeleri kaldır
    text = re.sub(r'\@w+|\#', '', text)
    # Noktalama işaretlerini kaldır
    text = re.sub(r'[^A-Za-z0-9ğüşöçİĞÜŞÖÇ\s]', '', text)
    # Küçük harfe çevir
    text = text.lower()
    return text

data['cleaned_text'] = data['text'].apply(clean_text)


In [14]:
# Tokenizer tanımlama
tokenizer = Tokenizer(num_words=5000, split=' ')
tokenizer.fit_on_texts(data['cleaned_text'].values)

X = tokenizer.texts_to_sequences(data['cleaned_text'].values)
X = pad_sequences(X)

# Etiketleri kategorik hale getirme
Y = pd.get_dummies(data['labels']).values


In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [16]:
# Modeli oluşturma
model = Sequential()
model.add(Embedding(5000, 128))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Modeli eğitme
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/10
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 559s 229ms/step - accuracy: 0.9471 - loss: 0.1667 - val_accuracy: 0.9575 - val_loss: 0.1217
Epoch 2/10
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 985s 405ms/step - accuracy: 0.9597 - loss: 0.1151 - val_accuracy: 0.9588 - val_loss: 0.1180
Epoch 3/10
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 635s 261ms/step - accuracy: 0.9651 - loss: 0.0991 - val_accuracy: 0.9598 - val_loss: 0.1172
Epoch 4/10
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 534s 219ms/step - accuracy: 0.9695 - loss: 0.0889 - val_accuracy: 0.9595 - val_loss: 0.1205
Epoch 5/10
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 525s 215ms/step - accuracy: 0.9721 - loss: 0.0818 - val_accuracy: 0.9597 - val_loss: 0.1222
Epoch 6/10
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 548s 225ms/step - accuracy: 0.9761 - loss: 0.0717 - val_accuracy: 0.9588 - val_loss: 0.1302
Epoch 7/10
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 540s 222ms/step - accuracy: 0.9778 - loss: 0.0654 - val_accuracy: 0.9574 - val_loss: 0.1341
Epoch 8/10
2435/2435 ━━━━━━━━━━━━━━━━━━━━ 646s 265ms/step - ac

In [19]:
# Modelin değerlendirilmesi
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=64)
print(f'Test accuracy: {acc}')


761/761 - 62s - 81ms/step - accuracy: 0.9583 - loss: 0.1483
Test accuracy: 0.9583367705345154


In [24]:
# Modeli native Keras formatında kaydetme
model.save('review_sentiment.keras')  # .h5 uzantısını kullanmıyoruz


In [25]:
from keras.models import load_model

# Modeli yükle
loaded_model = load_model('review_sentiment.keras')

# Modeli kullanarak tahmin yapabilirsiniz
score, acc = loaded_model.evaluate(X_test, Y_test, verbose=2, batch_size=64)
print(f'Test accuracy: {acc}')


C:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


761/761 - 34s - 45ms/step - accuracy: 0.9583 - loss: 0.1483
Test accuracy: 0.9583367705345154


In [ ]:
import re
import numpy as np
from keras.preprocessing.sequence import pad_sequences

# Kullanıcıdan alınan yorum
user_input = input("Yorumunuzu giriniz: ")

# Yorumu temizleme fonksiyonu (örnek olarak)
def clean_text(text):
    # Burada gerekli temizlik işlemlerini yapabilirsiniz
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # URL'leri kaldırma
    text = re.sub(r'\@w+|\#', '', text)  # İşaretlemeleri kaldırma
    text = re.sub(r'[^A-Za-zğüşöçİĞÜŞÖÇ\s]', '', text)  # Noktalama işaretlerini kaldırma
    text = text.lower()  # Küçük harfe çevirme
    return text

# Temizlenmiş metin
cleaned_input = clean_text(user_input)

# Tokenizer ve padding işlemi (eğitimdekiyle aynı tokenizasyon işlemleri)
max_len = 100  # Eğitimde belirlenen maksimum cümle uzunluğu
sequence = tokenizer.texts_to_sequences([cleaned_input])
padded_sequence = pad_sequences(sequence, maxlen=max_len)

# Tahmin yapma
prediction = loaded_model.predict(padded_sequence)
# Tahmin sonucunu etiketlere dönüştürme (örneğin)
labels = ['negatif', 'pozitif']
predicted_label = labels[np.argmax(prediction)]

print(f"Yorumun duygusu: {predicted_label}")
